<h1>
    Import Modules

In [ ]:
import string
import os
import pickle as pkl
from os import walk
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import tensorflow
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from livelossplot.tf_keras import PlotLossesCallback
import nltk
nltk.download('punkt')
import unicodedata
import joblib
from tensorflow import keras
import pickle

In [4]:
import sys
sys.path.append('/content/drive/MyDrive/Utils')

In [5]:
from data_wrangling import data_wrangling
from is_diacritic import is_diacritic
from is_char import is_char
from  split_daicrtics_from_arabic_text import split_daicrtics_from_arabic_text
from padding_sequence import padding_sequence
from concate_arabic_letters_astext_with_daicritic import concate_arabic_letters_astext_with_daicritic
from concate_arabic_letters_asnumber_with_daicritic import concate_arabic_letters_asnumber_with_daicritic
from remove_diacritics import remove_diacritics

In [6]:
ARABIC_LETTERS = frozenset([chr(x) for x in (list(range(0x0621, 0x63B)) + list(range(0x0641, 0x064B)))])
diacritic = ['َ', 'ً', 'ُ', 'ٌ', 'ِ', 'ٍ', 'ْ', 'ّ', 'َّ', 'ًّ', 'ُّ', 'ٌّ', 'ِّ', 'ٍّ','']

In [7]:
CHAR2INDEX = dict((l, n) for n, l in enumerate(sorted(ARABIC_LETTERS)))
CHAR2INDEX.update(dict((v, k) for k, v in enumerate([' '], len(CHAR2INDEX))))
print(CHAR2INDEX)

{'ء': 0, 'آ': 1, 'أ': 2, 'ؤ': 3, 'إ': 4, 'ئ': 5, 'ا': 6, 'ب': 7, 'ة': 8, 'ت': 9, 'ث': 10, 'ج': 11, 'ح': 12, 'خ': 13, 'د': 14, 'ذ': 15, 'ر': 16, 'ز': 17, 'س': 18, 'ش': 19, 'ص': 20, 'ض': 21, 'ط': 22, 'ظ': 23, 'ع': 24, 'غ': 25, 'ف': 26, 'ق': 27, 'ك': 28, 'ل': 29, 'م': 30, 'ن': 31, 'ه': 32, 'و': 33, 'ى': 34, 'ي': 35, ' ': 36}


In [8]:
DIAC2INDEX= {char: i for i, char in enumerate(diacritic)}
print(DIAC2INDEX)

{'َ': 0, 'ً': 1, 'ُ': 2, 'ٌ': 3, 'ِ': 4, 'ٍ': 5, 'ْ': 6, 'ّ': 7, 'َّ': 8, 'ًّ': 9, 'ُّ': 10, 'ٌّ': 11, 'ِّ': 12, 'ٍّ': 13, '': 14}


In [9]:
pd.set_option('max_colwidth', None)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', None)

<h1>
    Loading Tashkeela Corpus (Cleaned Version) from Huging Face

In [10]:
import datasets

<h1>
    Loading Train data and Convert it into DataFrame

In [11]:
train = datasets.load_dataset('arbml/tashkeelav2', split='train')


In [12]:
train = pd.DataFrame(train)


In [13]:
train.head()

,diacratized,text
0,إلَى مُوَكَّلِي الدَّائِنِ فَفِي حِكَايَتِهِ هَذِهِ إلْزَامُ الْمُوَكِّلِ الْمُتَوَفَّى بِالضَّمَانِ وَلِأَنَّ الدُّيُونَ تُقْضَى بِأَمْثَالِهَا إذْ يَثْبُتُ لِلْمَدِينِ فِي ذِمَّةِ الدَّائِنِ مِثْلُ مَطْلُوبِ الدَّائِنِ وَيَكُونُ تَقَاصَّا بَيْنَ الدَّيْنَيْنِ بِنَاءً عَلَيْهِ لَا يُقْبَلُ قَوْلُ الْوَكِيلِ هَذَا عَلَى الْمُوَكِّلِ بَعْدَ مَوْتِهِ وَيَلْزَمُ وَرَثَةَ الْمَدِينِ إثْبَاتُ دَفْعِهِ إلَى الدَّائِنِ وَتَسْلِيمِهِ إلَيْهِ وَاقِعَاتُ الْمُفْتِينَ وَالطَّحْطَاوِيُّ وَالْأَنْقِرْوِيُّ وَالْبَهْجَةُ,إلى موكلي الدائن ففي حكايته هذه إلزام الموكل المتوفى بالضمان ولأن الديون تقضى بأمثالها إذ يثبت للمدين في ذمة الدائن مثل مطلوب الدائن ويكون تقاصا بين الدينين بناء عليه لا يقبل قول الوكيل هذا على الموكل بعد موته ويلزم ورثة المدين إثبات دفعه إلى الدائن وتسليمه إليه واقعات المفتين والطحطاوي والأنقروي والبهجة
1,فَإِنْ أَبْرَءُوهُ بَعْدَ الْوَكَالَةِ لَمْ يَكُنْ وَكِيلًا فِي قَبْضِهِ أَيْضًا لِأَنَّ ذَلِكَ التَّوْكِيلَ كَانَ بَاطِلًا فَالْإِبْرَاءُ لَا يَنْقَلِبُ صَحِيحًا إلَّا أَنْ يُوَكِّلُوهُ بَعْدَ الْبَرَاءَةِ فَيَصِحُّ إنْشَاءُ التَّوْكِيلِ الْآنَ وَلَوْ كَانُوا أَبْرَءُوا الْعَبْدَيْنِ مِنْ دَيْنِهِمْ عَلَى أَنْ يَتَّبِعُوا الْمَوْلَى بِقَدْرِ الْقِيمَةِ مِنْ دَيْنِهِمْ وَتَرَاضَوْا عَلَى ذَلِكَ جَمِيعًا كَانَ جَائِزًا وَيَبْرَأُ الْعَبْدُ مِنْ الدُّيُونِ وَيَتَّبِعُونَ الْمَوْلَى بِالْقِيمَةِ لِأَنَّهُمْ بِهَذَا,فإن أبرءوه بعد الوكالة لم يكن وكيلا في قبضه أيضا لأن ذلك التوكيل كان باطلا فالإبراء لا ينقلب صحيحا إلا أن يوكلوه بعد البراءة فيصح إنشاء التوكيل الآن ولو كانوا أبرءوا العبدين من دينهم على أن يتبعوا المولى بقدر القيمة من دينهم وتراضوا على ذلك جميعا كان جائزا ويبرأ العبد من الديون ويتبعون المولى بالقيمة لأنهم بهذا
2,لِأَقْسَامٍ مِنْ الْعَرَبِيَّةِ جَعَلُوهَا أَيْ عُلَمَاءُ هَذَا الْعِلْمِ الْأَحْكَامَ الْمُسْتَنْبَطَةَ الْمَذْكُورَةَ مَادَّةً لَهُ أَيْ جُزْءًا لِهَذَا الْعِلْمِ وَإِنْ كَانَتْ هَذِهِ الْأَحْكَامُ فِي نَفْسِ الْأَمْرِ لَيْسَتْ مُدَوَّنَةً قَبْلَهُ أَيْ تَدْوِينِ هَذَا الْعِلْمِ وَإِنَّمَا تُذْكَرُ فِي غُضُونِ اسْتِدْلَالَاتِهِمْ فِي الْفُرُوعِ وَغَيْرِهَا وَذَلِكَ كَالْعُمُومِ وَالْخُصُوصِ وَالتَّبَايُنِ وَالتَّرَادُفِ وَالْحَقِيقَةِ وَالْمَجَازِ وَالظُّهُورِ وَالنُّصُوصِيَّةِ وَالْإِشَارَةِ وَالْعِبَارَةِ,لأقسام من العربية جعلوها أي علماء هذا العلم الأحكام المستنبطة المذكورة مادة له أي جزءا لهذا العلم وإن كانت هذه الأحكام في نفس الأمر ليست مدونة قبله أي تدوين هذا العلم وإنما تذكر في غضون استدلالاتهم في الفروع وغيرها وذلك كالعموم والخصوص والتباين والترادف والحقيقة والمجاز والظهور والنصوصية والإشارة والعبارة
3,بِالْأَشْيَاءِ كَوْنُهَا وَيُسَمَّى هَذَا عِلْمًا بِمَا سَيَكُونُ ثُمَّ يَعْلَمُ بَعْدَ كَوْنِهَا أَنَّهَا كَانَتْ وَذَلِكَ عِلْمٌ بِمَا كَانَ وَالْعِلْمُ بِمَا سَيَكُونُ غَيْرُ الْعِلْمِ بِمَا كَانَ وَرُدَّ بِأَنَّ التَّعْبِيرَ بِمَا كَانَ أَوْ سَيَكُونُ بِاعْتِبَارِ الْمَعْلُومِ لَا بِاعْتِبَارِ الْعِلْمِ وَتَعَلُّقِهِ فَإِنَّهُ وَاحِدٌ فَالْمَعْلُومُ قَبْلَ كَوْنِهِ يُعَبَّرُ عَنْهُ بِأَنَّهُ سَيَكُونُ وَبَعْدَ كَوْنِهِ يُعَبَّرُ عَنْهُ بِأَنَّهُ كَانَ وَمَثَّلَهُ الشِّهْرِسْتَانِيّ بِمَا لَوْ أَخْبَرَنَا,بالأشياء كونها ويسمى هذا علما بما سيكون ثم يعلم بعد كونها أنها كانت وذلك علم بما كان والعلم بما سيكون غير العلم بما كان ورد بأن التعبير بما كان أو سيكون باعتبار المعلوم لا باعتبار العلم وتعلقه فإنه واحد فالمعلوم قبل كونه يعبر عنه بأنه سيكون وبعد كونه يعبر عنه بأنه كان ومثله الشهرستاني بما لو أخبرنا
4,فَمَنْ تَعَجَّلَ فِي يَوْمَيْنِ فَلَا إِثْمَ عَلَيْهِ وَمَنْ تَأَخَّرَ فَلَا إِثْمَ عَلَيْهِ ثُمَّ أَرْدَفَ خَلْفَهُ فَجَعَلَ يُنَادِي بِهِنَّ حَدِيثُ بِشْرِ بْنِ سُحَيْمٍ رَضِيَ اللَّهُ تَعَالَى عَنْهُ حَدَّثَنَا سُرَيْجٌ حَدَّثَنَا حَمَّادٌ يَعْنِي ابْنَ زَيْدٍ عَنْ عَمْرِو بْنِ دِينَارٍ عَنْ نَافِعِ بْنِ جُبَيْرٍ عَنْ بِشْرِ بْنِ سُحَيْمٍ أَنَّ النَّبِيَّ صَلَّى اللَّهُ عَلَيْهِ وَسَلَّمَ أَمَرَ أَنْ يُنَادَى أَيَّامَ التَّشْرِيقِ أَنَّهُ لَا يَدْخُلُ الْجَنَّةَ إِلَّا مُؤْمِنٌ وَهِيَ أَيَّامُ أَكْلٍ وَشُرْبٍ,فمن تعجل في يومين فلا إثم عليه ومن تأخر فلا إثم عليه ثم أردف خلفه فجعل ينادي بهن حديث بشر بن سحيم رضي الله تعالى عنه حدثنا سريج حدثنا حماد يعني ابن زيد عن عمرو بن دينار

In [14]:
train.tail()

,diacratized,text
522458,وَلَنَا مَا رُوِيَ أَنَّ النَّبِيَّ صَلَّى اللَّهُ عَلَيْهِ وَسَلَّمَ صَلَّى عَلَى جِنَازَةٍ فَلَمَّا فَرَغَ جَاءَ عُمَرُ وَمَعَهُ قَوْمٌ فَأَرَادَ أَنْ يُصَلِّيَ ثَانِيًا فَقَالَ لَهُ النَّبِيُّ صَلَّى اللَّهُ عَلَيْهِ وَسَلَّمَ الصَّلَاةُ عَلَى الْجِنَازَةِ لَا تُعَادُ وَلَكِنْ اُدْعُ لِلْمَيِّتِ وَاسْتَغْفِرْ لَهُ وَهَذَا نَصٌّ فِي الْبَابِ وَرُوِيَ أَنَّ ابْنَ عَبَّاسٍ وَابْنَ عُمَرَ رَضِيَ اللَّهُ تَعَالَى عَنْهُمْ فَاتَتْهُمَا صَلَاةٌ عَلَى جِنَازَةٍ فَلَمَّا حَضَرَا مَا زَادَا عَلَى الِاسْتِغْفَارِ لَهُ,ولنا ما روي أن النبي صلى الله عليه وسلم صلى على جنازة فلما فرغ جاء عمر ومعه قوم فأراد أن يصلي ثانيا فقال له النبي صلى الله عليه وسلم الصلاة على الجنازة لا تعاد ولكن ادع للميت واستغفر له وهذا نص في الباب وروي أن ابن عباس وابن عمر رضي الله تعالى عنهم فاتتهما صلاة على جنازة فلما حضرا ما زادا على الاستغفار له
522459,وَهِيَ أَيْ صَدَقَةُ الْفِطْرِ نِصْفُ صَاعٍ مِنْ بُرٍّ أَوْ دَقِيقِهِ أَوْ سَوِيقِهِ وَالْمُرَادُ مِنْهُمَا مَا يُتَّخَذُ مِنْ الْبُرِّ أَمَّا دَقِيقُ الشَّعِيرِ أَوْ سَوِيقُهُ فَكَالشَّعِيرِ وَالْأَوْلَى أَنْ يُرَاعَى فِيهِمَا الْقَدْرُ وَالْقِيمَةُ أَوْ صَاعٌ مِنْ تَمْرٍ وَشَعِيرٍ لِقَوْلِهِ عَلَيْهِ الصَّلَاةُ وَالسَّلَامُ أَدُّوا عَنْ كُلِّ حُرٍّ وَعَبْدٍ صَغِيرٍ أَوْ كَبِيرٍ نِصْفَ صَاعٍ مِنْ بُرٍّ أَوْ صَاعًا مِنْ تَمْرٍ أَوْ صَاعًا مِنْ شَعِيرٍ وَهَذَا حُجَّةٌ عَلَى الشَّافِعِيِّ فَإِنَّهُ قَالَ فِي الْكُلِّ,وهي أي صدقة الفطر نصف صاع من بر أو دقيقه أو سويقه والمراد منهما ما يتخذ من البر أما دقيق الشعير أو سويقه فكالشعير والأولى أن يراعى فيهما القدر والقيمة أو صاع من تمر وشعير لقوله عليه الصلاة والسلام أدوا عن كل حر وعبد صغير أو كبير نصف صاع من بر أو صاعا من تمر أو صاعا من شعير وهذا حجة على الشافعي فإنه قال في الكل
522460,قَوْلُهُ الْمُقْتَضَى أَوْ الْمُخَيَّرُ الْإِسْنَادُ فِيهِمَا مَجَازِيٌّ إذْ الْمُقْتَضَى وَالْمُخَيَّرُ فِي الْحَقِيقَةِ هُوَ اللَّهُ قَوْلُهُ كَمَا تَقَدَّمَ أَيْ عِنْدَ قَوْلِ الْمُصَنِّفِ وَالْحُكْمُ خِطَابُ اللَّهِ وَهُوَ رَاجِعٌ لِقَوْلِهِ الَّذِي هُوَ الْحُكْمُ الْمُتَعَارَفُ أَيْ مِنْ أَنَّهُ خِطَابٌ مُتَعَلِّقٌ بِفِعْلِ الْمُكَلَّفِ مِنْ حَيْثُ إنَّهُ مُلْزَمٌ مَا فِيهِ كُلْفَةٌ قَوْلُهُ وَمِنْ خِطَابِ الْوَضْعِ نَبَّهَ بِتَكْرِيرِ مِنْ عَلَى أَنَّ مَقْصُودَ الْمُصَنِّفِ بِالسُّنَّةِ لِلْوَضْعِ خِطَابُ,قوله المقتضى أو المخير الإسناد فيهما مجازي إذ المقتضى والمخير في الحقيقة هو الله قوله كما تقدم أي عند قول المصنف والحكم خطاب الله وهو راجع لقوله الذي هو الحكم المتعارف أي من أنه خطاب متعلق بفعل المكلف من حيث إنه ملزم ما فيه كلفة قوله ومن خطاب الوضع نبه بتكرير من على أن مقصود المصنف بالسنة للوضع خطاب
522461,طَلَبَهُ وَامْتَنَعَ الْمُشْتَرِي مِنْهُ لِأَنَّ الْمُشْتَرِيَ قَدْ يَكُونُ غَرَضُهُ فِي الْجَمِيعِ وَمِنْهُ مَا يَأْخُذُهُ الشَّفِيعُ قَوْلُهُ غَيْرُ مُعْتَبَرٍ أَيْ بَلْ لَوْ كَانَتْ الْحِصَّةُ وَاحِدَةً وَأَرَادَ الشَّفِيعُ أَخْذَ بَعْضِهَا بِالشُّفْعَةِ لَمْ يُجْبَرْ الْمُشْتَرِي عَلَى التَّبْعِيضِ وَكَذَلِكَ إذَا تَعَدَّدَتْ الْحِصَصُ وَكَانَ بَائِعُهَا وَاحِدًا كَمَا لَوْ كَانَتْ دَارٌ وَحَانُوتٌ وَبُسْتَانٌ شَرِكَةً بَيْنَ اثْنَيْنِ وَبَاعَ أَحَدُهُمَا حِصَّتَهُ فِي الثَّلَاثَةِ لِأَجْنَبِيٍّ فَلَيْسَ,طلبه وامتنع المشتري منه لأن المشتري قد يكون غرضه في الجميع ومنه ما يأخذه الشفيع قوله غير معتبر أي بل لو كانت الحصة واحدة وأراد الشفيع أخذ بعضها بالشفعة لم يجبر المشتري على التبعيض وكذلك إذا تعددت الحصص وكان بائعها واحدا كما لو كانت دار وحانوت وبستان شركة بين اثنين وباع أحدهما حصته في الثلاثة لأجنبي فليس
522462,قُلْتُ فَإِنْ كَانَ أَرَادَ أَنْ يَرُدَّ قَالَ مَالِكٌ ذَلِكَ لَهُ فَإِنْ قَالَ الْبَائِعُ لَيْسَ ذَلِكَ لَكَ حَتَّى تَسْتَشِيرَ فُلَانًا لَمْ يُلْتَفَتْ إلَى قَوْلِ الْبَائِعِ قَالَ مَالِكٌ وَاَلَّذِي اشْتَرَى عَلَى إنْ رَضِيَ فُلَانٌ فَلَيْسَ ذَلِكَ لِلْمُشْتَرِي أَنْ يُمْضِيَ وَلَا يَرُدَّ حَتَّى يَرْضَى فُلَانٌ الَّذِي جَعَلَ لَهُ الرِّضَا وَاَلَّذِي اشْتَرَى عَلَى أَنَّ فُلَانًا بِالْخِيَارِ مِثْلُ ذَلِكَ وَقَالَ أَشْهَبُ إنَّهُ جَائِزٌ إذَا اشْتَرَى سِلْعَةً عَلَى أَنَّ رَجُلًا أَجْنَبِيًّا أَوْ ذَا قَرَابَةٍ,قلت فإن كان أراد أن يرد قال مالك ذلك له فإن قال البائع ليس ذلك لك حتى تستشير فلانا لم يلتفت إلى قول البائع قال مالك والذي اشترى على إ

In [15]:
# Checking Length of diacratized text column in dataFrame
diacritic_train = train['diacratized']
len(diacritic_train)

522463

<h1>
    Loading Test data & Convert it into DataFrame

In [16]:
test = datasets.load_dataset('arbml/tashkeelav2', split='test')

In [17]:
test = pd.DataFrame(test)

In [18]:
test.head()

,diacratized,text
0,قَالَ وَإِنْ كَانَ فِيهَا شَيْءٌ بَارٌّ لَا يَجُوزُ فَإِنَّهُ حَانِثٌ قُلْتُ أَرَأَيْتَ إنْ حَلَفَ رَجُلٌ لِغَرِيمٍ لَهُ أَنْ لَا يُفَارِقَهُ حَتَّى يَسْتَوْفِيَ مِنْهُ حَقَّهُ فَأَخَذَ مِنْهُ حَقَّهُ فَلَمَّا افْتَرَقَا أَصَابَ بَعْضَهَا نُحَاسًا أَوْ رَصَاصًا أَوْ نَقْصًا بَيِّنٌ نُقْصَانُهَا أَيَحْنَثُ فِي قَوْلِ مَالِكٍ أَمْ لَا قَالَ هُوَ حَانِثٌ لِأَنِّي سَأَلْت مَالِكًا عَنْ الرَّجُلِ يَحْلِفُ بِطَلَاقِ امْرَأَتِهِ لَيَقْضِيَنَّهُ حَقَّهُ إلَى أَجَلٍ فَيَقْضِيه حَقَّهُ ثُمَّ يَذْهَبُ صَاحِبُ الْحَقِّ,قال وإن كان فيها شيء بار لا يجوز فإنه حانث قلت أرأيت إن حلف رجل لغريم له أن لا يفارقه حتى يستوفي منه حقه فأخذ منه حقه فلما افترقا أصاب بعضها نحاسا أو رصاصا أو نقصا بين نقصانها أيحنث في قول مالك أم لا قال هو حانث لأني سألت مالكا عن الرجل يحلف بطلاق امرأته ليقضينه حقه إلى أجل فيقضيه حقه ثم يذهب صاحب الحق
1,قِيَامِ الْمُسْتَحِقِّ فَقَالَ ابْنُ الْمَوَّازِ لَا شَيْءَ عَلَى الْمُشْتَرِي فِي قَوْلِ مَالِكٍ الَّذِي قَالَ فِيهِ يَأْخُذُ قِيمَتَهَا فَقَطْ إذَا وَجَدَهَا حَيَّةً وَهُوَ ثَالِثُ الْأَقْوَالِ وَكَلَامُ الْمُصَنِّفِ فِي الْوَلَدِ الَّذِي وُجِدَ حَيًّا وَأَمَّا لَوْ مَاتَ وَلَدُهَا حَتْفَ أَنْفِهِ قَبْلَ الِاسْتِحْقَاقِ فَلَا شَيْءَ فِيهِ وَأَمَّا لَوْ قُتِلَ عَمْدًا فَلَا شَيْءَ فِيهِ عَلَى الْأَبِ إنْ اُقْتُصَّ مِنْ قَاتِلِهِ أَوْ عُفِيَ عَنْهُ لَكِنْ لِلْمُسْتَحِقِّ الرُّجُوعُ عَلَى الْقَاتِلِ بِالْأَقَلِّ,قيام المستحق فقال ابن المواز لا شيء على المشتري في قول مالك الذي قال فيه يأخذ قيمتها فقط إذا وجدها حية وهو ثالث الأقوال وكلام المصنف في الولد الذي وجد حيا وأما لو مات ولدها حتف أنفه قبل الاستحقاق فلا شيء فيه وأما لو قتل عمدا فلا شيء فيه على الأب إن اقتص من قاتله أو عفي عنه لكن للمستحق الرجوع على القاتل بالأقل
2,فَقَدْ اجْتَهَدَ الْإِمَامُ الشَّافِعِيُّ وَالْإِمَامُ مُحَمَّدٌ وَالْإِمَامُ مَالِكٌ رَحِمَهُمْ اللَّهُ تَعَالَى بِعَدَمِ لُزُومِ قَبُولِ الْمُحَالِ عَلَيْهِ وَرِضَاهُ وَبِإِمْكَانِ عَقْدِ الْحَوَالَةِ بَيْنَ الْمُحِيلِ وَالْمُحَالِ لَهُ لِأَنَّ الْحَقَّ لِلْمُحِيلِ فَكَمَا أَنَّهُ مُقْتَدِرٌ عَلَى اسْتِيفَائِهِ بِذَاتِهِ فَهُوَ مُقْتَدِرٌ أَيْضًا عَلَى اسْتِيفَائِهِ بِوَاسِطَةِ غَيْرِهِ وَلَنَا أَنَّ الْحَوَالَةَ تُصْرَفُ عَلَى الْمُحَالِ عَلَيْهِ بِنَقْلِ الْحَقِّ إلَى ذِمَّتِهِ فَلَا يَتِمُّ إلَّا بِقَبُولِهِ,فقد اجتهد الإمام الشافعي والإمام محمد والإمام مالك رحمهم الله تعالى بعدم لزوم قبول المحال عليه ورضاه وبإمكان عقد الحوالة بين المحيل والمحال له لأن الحق للمحيل فكما أنه مقتدر على استيفائه بذاته فهو مقتدر أيضا على استيفائه بواسطة غيره ولنا أن الحوالة تصرف على المحال عليه بنقل الحق إلى ذمته فلا يتم إلا بقبوله
3,فِيمَنْ أَنْفَقَ عَلَى ضَالَّةٍ وَفِي الْوَصِيِّ يَرْهَنُ لِلْيَتِيمِ رَهْنًا مِنْ مَالِ الْيَتِيمِ قُلْتُ أَرَأَيْتَ الضَّالَّةَ أَلَيْسَ لَهُ مَا أَنْفَقَ عَلَيْهَا وَلَيْسَ لِصَاحِبِهَا أَنْ يَأْخُذَهَا حَتَّى يُعْطِيَهُ نَفَقَتَهَا فِي قَوْلِ مَالِكٍ وَيَكُونُ أَوْلَى بِهَا مِنْ الْغُرَمَاءِ حَتَّى يَقْبِضَ نَفَقَتَهُ وَقَدْ أَنْفَقَ بِغَيْرِ أَمْرِ رَبِّهَا فَلِمَ قَالَ مَالِكٌ فِي الضَّالَّةِ هُوَ أَوْلَى بِهَا وَفِي نَفَقَةِ الرَّاهِنِ لِمَ لَا يَكُونُ أَوْلَى بِهَا أَيْضًا قَالَ لِأَنَّ الضَّالَّةَ لَا,فيمن أنفق على ضالة وفي الوصي يرهن لليتيم رهنا من مال اليتيم قلت أرأيت الضالة أليس له ما أنفق عليها وليس لصاحبها أن يأخذها حتى يعطيه نفقتها في قول مالك ويكون أولى بها من الغرماء حتى يقبض نفقته وقد أنفق بغير أمر ربها فلم قال مالك في الضالة هو أولى بها وفي نفقة الراهن لم لا يكون أولى بها أيضا قال لأن الضالة لا
4,أَخْبَرَنَا أَبُو عَبْدِ اللَّهِ الْحَافِظُ أَخْبَرَنِى أَحْمَدُ بْنُ سَهْلٍ الْفَقِيهُ حَدَّثَنَا إِبْرَاهِيمُ بْنُ مَعْقِلٍ حَدَّثَنَا حَرْمَلَةُ حَدَّثَنَا ابْنُ وَهْبٍ حَدَّثَنِى مَالِكٌ أَنَّ أَبَانَ بْنَ عُثْمَانَ حِينَ وَلِىَ الْمَدِينَةَ فِى خِلاَفَةِ عَبْدِ الْمَلِكِ بْنِ مَرْوَانَ فَأَرَادَ أَنْ يَنْقُضَ مَا كَانَ عَبْدُ اللَّهِ بْنُ الزُّبَيْرِ قَضَى فِيهِ فَكَتَبَ أَبَانُ بْنُ عُثْمَانَ فِى ذَلِكَ إِلَى عَبْدِ الْمَلِكِ فَكَتَبَ إِلَيْهِ عَبْدُ الْمَلِكِ إِنَّا لَمْ نَنْقَمْ عَلَى ابْنِ الزُّبَيْرِ,أخبرنا أبو عبد الله الحافظ أخبرنى أحمد بن سهل الفقيه حدثنا إبراهيم بن معقل حدثنا حرملة حدثنا ابن وهب حدثنى مالك أن أبان بن عثمان حين ولى المدينة فى خلافة عبد ا

In [19]:
test.tail()

,diacratized,text
58047,أَنَّ مَا نَقَلَهُ الشَّارِحُ أَوَّلًا عَنْ الْفَتْحِ مِنْ أَنَّ الصَّرِيحَ مَا لَا يَحْتَاجُ إلَى نِيَّةٍ بَائِنًا كَانَ الْوَاقِعُ بِهِ أَوْ رَجْعِيًّا خَاصٌّ بِالصَّرِيحِ فِي الْجُمْلَةِ الْأُولَى أَعْنِي قَوْلَهُمْ الصَّرِيحُ يَلْحَقُ الصَّرِيحَ وَالْبَائِنَ كَمَا دَلَّ عَلَيْهِ كَلَامُ الْفَتْحِ الَّذِي ذَكَرْنَاهُ هُنَا وَيَدُلُّ عَلَيْهِ أَيْضًا أُمُورٌ مِنْهَا مَا أَطْبَقُوا عَلَيْهِ مِنْ تَعْلِيلِهِمْ عَدَمَ لُحُوقِ الْبَائِنِ الْبَائِنَ بِإِمْكَانِ جَعْلِ الثَّانِي خَبَرًا عَنْ الْأَوَّلِ وَلَا يَخْفَى,أن ما نقله الشارح أولا عن الفتح من أن الصريح ما لا يحتاج إلى نية بائنا كان الواقع به أو رجعيا خاص بالصريح في الجملة الأولى أعني قولهم الصريح يلحق الصريح والبائن كما دل عليه كلام الفتح الذي ذكرناه هنا ويدل عليه أيضا أمور منها ما أطبقوا عليه من تعليلهم عدم لحوق البائن البائن بإمكان جعل الثاني خبرا عن الأول ولا يخفى
58048,مَنْ يَدْخُلُ الْجَنَّةَ يَوْمَ الْقِيَامَةِ وَلَا فَخْرَ وَإِنِّي آتِي بَابَ الْجَنَّةِ فَآخُذُ بِحَلْقَتِهَا فَيَقُولُونَ مَنْ هَذَا فَيَقُولُ أَنَا مُحَمَّدٌ فَيَفْتَحُونَ لِي فَأَدْخُلُ فَإِذَا الْجَبَّارُ عَزَّ وَجَلَّ مُسْتَقْبِلِي فَأَسْجُدُ لَهُ فَيَقُولُ ارْفَعْ رَأْسَكَ يَا مُحَمَّدُ وَتَكَلَّمْ يُسْمَعْ مِنْكَ وَقُلْ يُقْبَلْ مِنْكَ وَاشْفَعْ تُشَفَّعْ فَأَرْفَعُ رَأْسِي فَأَقُولُ أُمَّتِي أُمَّتِي يَا رَبِّ فَيَقُولُ اذْهَبْ إِلَى أُمَّتِكَ فَمَنْ وَجَدْتَ فِي قَلْبِهِ مِثْقَالَ حَبَّةٍ مِنْ شَعِيرٍ,من يدخل الجنة يوم القيامة ولا فخر وإني آتي باب الجنة فآخذ بحلقتها فيقولون من هذا فيقول أنا محمد فيفتحون لي فأدخل فإذا الجبار عز وجل مستقبلي فأسجد له فيقول ارفع رأسك يا محمد وتكلم يسمع منك وقل يقبل منك واشفع تشفع فأرفع رأسي فأقول أمتي أمتي يا رب فيقول اذهب إلى أمتك فمن وجدت في قلبه مثقال حبة من شعير
58049,فَإِنْ احْتَجَمَ فَظَنَّ أَنَّ ذَلِكَ يُفْطِرُهُ فَأَكَلَ بَعْدَ ذَلِكَ مُتَعَمِّدًا إنْ اسْتَفْتَى فَقِيهًا فَأَفْتَاهُ بِأَنَّهُ قَدْ أَفْطَرَ فَلَا كَفَّارَةَ عَلَيْهِ لِأَنَّ الْعَامِّيَّ يَلْزَمُهُ تَقْلِيدُ الْعَالِمِ فَكَانَتْ الشُّبْهَةُ مُسْتَنِدَةً إلَى صُورَةِ دَلِيلٍ وَإِنْ بَلَغَهُ خَبَرُ الْحِجَامَةِ وَهُوَ الْمَرْوِيُّ عَنْ رَسُولِ اللَّهِ صَلَّى اللَّهُ عَلَيْهِ وَسَلَّمَ أَفْطَرَ الْحَاجِمُ وَالْمَحْجُومُ رَوَى الْحَسَنُ عَنْ أَبِي حَنِيفَةَ أَنَّهُ لَا كَفَّارَةَ عَلَيْهِ لِأَنَّ ظَاهِرَ الْحَدِيثِ,فإن احتجم فظن أن ذلك يفطره فأكل بعد ذلك متعمدا إن استفتى فقيها فأفتاه بأنه قد أفطر فلا كفارة عليه لأن العامي يلزمه تقليد العالم فكانت الشبهة مستندة إلى صورة دليل وإن بلغه خبر الحجامة وهو المروي عن رسول الله صلى الله عليه وسلم أفطر الحاجم والمحجوم روى الحسن عن أبي حنيفة أنه لا كفارة عليه لأن ظاهر الحديث
58050,الْقَاعِدَتَيْنِ وَذَلِكَ أَنَّ الْقَائِلَ الْأَوَّلَ يَقُولُ غُرُوبُ الشَّمْسِ مِنْ يَوْمِ الصَّوْمِ سَبَبٌ وَمَا بَعْدَهُ ظَرْفٌ لِلتَّكْلِيفِ فَقَطْ وَلَا يَكُونُ شَيْءٌ مِنْ أَجْزَاءِ هَذَا الزَّمَانِ سَبَبًا لِلتَّكْلِيفِ وَالْقَائِلُ الرَّابِعُ يَقُولُ كُلُّ جُزْءٍ مِنْ أَجْزَاءِ هَذَا الزَّمَانِ مِنْ الْغُرُوبِ إلَى الْغُرُوبِ ظَرْفٌ لِلتَّكْلِيفِ وَسَبَبٌ لَهُ فَقَدْ اشْتَرَكَا فِي التَّوْسِعَةِ لَكِنْ تَوْسِعَةُ الْأَوَّلِ كَتَوْسِعَةِ قَضَاءِ رَمَضَانَ وَتَوْسِعَةُ الثَّانِي كَتَوْسِعَةِ صَلَاةِ الظُّهْرِ,القاعدتين وذلك أن القائل الأول يقول غروب الشمس من يوم الصوم سبب وما بعده ظرف للتكليف فقط ولا يكون شيء من أجزاء هذا الزمان سببا للتكليف والقائل الرابع يقول كل جزء من أجزاء هذا الزمان من الغروب إلى الغروب ظرف للتكليف وسبب له فقد اشتركا في التوسعة لكن توسعة الأول كتوسعة قضاء رمضان وتوسعة الثاني كتوسعة صلاة الظهر
58051,الْآيَةُ الْخَامِسَةُ وَالسِّتُّونَ قَوْله تَعَالَى لِلَّذِينَ يُؤْلُونَ مِنْ نِسَائِهِمْ تَرَبُّصُ أَرْبَعَةِ أَشْهُرٍ فَإِنْ فَاءُوا فَإِنَّ اللَّهَ غَفُورٌ رَحِيمٌ وَإِنْ عَزَمُوا الطَّلَاقَ فَإِنَّ اللَّهَ سَمِيعٌ عَلِيمٌ فِيهَا سِتَّ عَشْرَةَ مَسْأَلَةً الْمَسْأَلَةُ الْأُولَى فِي سَبَبِ نُزُولِهَا وَهِيَ آيَةٌ عَظِيمَةُ الْمَوْقِعِ جِدًّا يَتَرَتَّبُ عَلَيْهَا حُكْمٌ كَبِيرٌ اخْتَلَفَ فِيهِ الصَّحَابَةُ وَالتَّابِعُونَ وَفُقَهَاءُ الْأَمْصَارِ وَدَقَّتْ مَدَارِكُهَا حَسْبَمَا تَرَوْنَهَا مِنْ جُمْلَتِهَا,الآية الخامسة والستون قوله تعالى للذين يؤلون من نسائهم تربص أربعة أشهر فإن فاءوا فإن الله غفور رحيم وإن عزموا الطلاق فإن الله سم

In [20]:
# Checking Length of diacratized text column in dataFrame
diacritic_test = test['diacratized']
len(diacritic_test)

58052

# Data Cleaning & Preprocessing

In [21]:
# import inspect
# # se inspect.getsource to view the source code of the function
# source_code = inspect.getsource(data_wrangling)
# print(source_code)

# Wrangling and Retrieve Cleaned Train DataFrame

In [22]:
train['diacratized']  = train['diacratized'].apply(data_wrangling)

# Wrangling and Retrieve Cleaned Test DataFrame

In [23]:
test['diacratized']  = test['diacratized'].apply(data_wrangling)

# Taking A Part of DataFrame

In [24]:
train_part = train.iloc[:60000, :]
test_part  = test.iloc[:6000,:]
print('len_train_part = ',len(train_part))
print('len_test_part = ',len(test_part))

len_train_part =  60000
len_test_part =  6000


# Applying Split Daicrtics From Arabictext

In [ ]:
train_part[['X' ,'y']] = train_part['diacratized'].apply(lambda x: pd.Series(split_daicrtics_from_arabic_text(x)))
test_part[['X' ,'y']] = test_part['diacratized'].apply(lambda x: pd.Series(split_daicrtics_from_arabic_text(x)))

# Applaying Padding

In [27]:
max_length = 100
X , y = padding_sequence(max_length , train_part['X'] ,len(CHAR2INDEX), train_part['y'] ,len(diacritic)-1 )

# Applaying Train validation Split

In [42]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.04, random_state=42)

# Saving Train and Test

In [41]:
# Specify the directory path to save the files
save_directory = '/content/drive/MyDrive/Utils'

# Create the directory if it doesn't exist
os.makedirs(save_directory, exist_ok=True)

# Save x_train
with open(os.path.join(save_directory, 'X_train.pkl'), 'wb') as f:
    pickle.dump(x_train, f)

# Save y_train
with open(os.path.join(save_directory, 'y_train.pkl'), 'wb') as f:
    pickle.dump(y_train, f)

# Save x_test
with open(os.path.join(save_directory, 'X_val.pkl'), 'wb') as f:
    pickle.dump(x_val, f)

# Save y_test
with open(os.path.join(save_directory, 'y_val.pkl'), 'wb') as f:
    pickle.dump(y_val, f)